# Pipeline
 This pipeline should pipe an image through the localization model and the recognition model to first localize a sign in the image and then decide which sign it is. 

## Dependencies

At first, we have to install  and import some libraries 

In [23]:
import dlib
# vs_build_tools heruntergeladen und installiert, dabei habe ich c++ ausgewählt
# dann pip install dlib

ModuleNotFoundError: No module named 'dlib'

## Import the image

To import your own images, edit the path of the example_images_folder.

In [3]:
example_images_folder = "images"


To be able to work with the data, convert .ppm into .jpg

In [4]:
import os
#pip install opencv-python
import cv2
# pip install numpy
import numpy as np

__current_directory = os.getcwd()
dataset_path = os.path.join(__current_directory, example_images_folder)

for file in os.listdir(dataset_path):
        if not file.endswith(".ppm"):
            continue
        el_image = cv2.imread(os.path.join(dataset_path, file))
        cv2.imwrite(os.path.join(dataset_path, file.replace(".ppm", ".jpg")), el_image)

# Localization

## Load the models 

In [ ]:
models_directory = os.path.join(os.getcwd(), "../localization/resources", "models")
detectors = {
    model[6:-4]: dlib.simple_object_detector(os.path.join(models_directory, model))
    for model in os.listdir(models_directory)
}


images_folder = os.path.join(os.getcwd(), "FullIJCNN2013")
confidence_threshold = 0.5  # Confidence threshold to display detections

# Adjusted text for readability
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7  # Larger scale for better readability
font_thickness = 2  # Thicker text for better visibility

img = cv2.imread(os.path.join(dataset_path, "00000.png"))

# Assuming 'boxes', 'confidences', 'detector_idxs' are returned from your model

boxes, confidences, detector_idxs = dlib.simple_object_detector.run_multiple(
    list(detectors.values()), img, upsample_num_times=1, adjust_threshold=0.0
)
annotation_position = (img.shape[1] - 250, 30)  # Position near the top-right corner
cv2.putText(img, "custom_annotation", annotation_position, font, font_scale, (0, 0, 255), font_thickness)

for i in range(len(boxes)):
    if confidences[i] < confidence_threshold:
        continue  # Skip if confidence is below threshold
    
    box = boxes[i]
    label = f"Detector {list(detectors.keys())[detector_idxs[i]]}: {confidences[i]:.2f}"

    # Draw bounding box
    cv2.rectangle(img, (box.left(), box.top()), (box.right(), box.bottom()), (0, 0, 255), 2)

    # Display label and confidence below the box
    text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
    text_x = box.left()
    text_y = box.bottom() + text_size[1] + 5

    cv2.putText(img, label, (text_x, text_y), font, font_scale, (0, 0, 255), font_thickness)

# Display the image with annotations
cv2.imshow("Annotated Image", img)

NameError: name 'dlib' is not defined

In [ ]:


cv2.destroyAllWindows()

## Recognition

### Prepare the image

In [9]:
img = cv2.imread("./images/00000.jpg")
img = cv2.resize(img, (64, 64))
img = np.expand_dims(img, axis=0)

## Load the model

In [10]:
import tensorflow as tf
# Load the model
model = tf.keras.models.load_model('../recognition/final_model.h5', compile=False)

c:\Users\alisa\Desktop\Ki Projekt\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Predict the sign

In [11]:

predictions = model.predict(img)
predicted_class_id = np.argmax(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step


## Use the output

In [ ]:
import sys
sys.path.append("../recognition/")
from constants import CLASS_ID_TO_NAME

predicted_class_name = CLASS_ID_TO_NAME.get(predicted_class_id, "Unknown Class")

print(predicted_class_name)

Keep right
